In [1]:
import numpy as np
import os
import mediapipe as mp
import cv2
import keyboard

from tensorflow.keras.models import load_model

2023-08-10 17:24:11.544624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
pip install flask-cors

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for flask-cors from https://files.pythonhosted.org/packages/10/69/1e6cfb87117568a9de088c32d6258219e9d1ff7c131abf74249ef2031279/Flask_Cors-4.0.0-py2.py3-none-any.whl.metadata
Using cached Flask_Cors-4.0.0-py2.py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [109]:
import os

def load_labels(image_dir):
    labels = []
    for dirpath, _, filenames in os.walk(image_dir):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                label = os.path.basename(os.path.dirname(os.path.join(dirpath, filename)))
                labels.append(label)
    return labels

In [111]:
x = load_labels('/Users/samannazir/Downloads/archive/asl_alphabet_train/asl_alphabet_train')

x

['R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R',
 'R'

In [2]:
# Set the path to the data directory
PATH = os.path.join('/Users/samannazir/Downloads/archive/asl_alphabet_train/asl_alphabet_train')

# Load labels from the data directory using the load_labels function
#labels = load_labels(PATH)

# Convert the labels list to a numpy array to create the actions array
actions = np.array( ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
           'Space', 'Nothing', 'Delete']
)

# Load the trained model
model = load_model('ASL_model')

In [33]:
actions

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'Space', 'Nothing', 'Delete'], dtype='<U7')

In [ ]:
#////////////////

In [9]:
import cv2
import numpy as np
import os
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR

    return image, results

def draw_styled_landmarks(image, results):
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())


def extract_keypoints(results):
    keypoints_list = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            if hand_landmarks:  # Check if hand landmarks are present
                keypoints = np.array([[res.x, res.y, res.z] for res in hand_landmarks.landmark])
                # Resize keypoints to match the model's input shape
                keypoints_resized = cv2.resize(keypoints, (100, 100), interpolation=cv2.INTER_AREA)
                keypoints_resized_with_channels = np.expand_dims(keypoints_resized, axis=-1)
                keypoints_resized_with_channels = np.repeat(keypoints_resized_with_channels, 3, axis=-1)
                keypoints_list.append(keypoints_resized_with_channels)
            else:
                keypoints_list.append(np.zeros((100, 100, 3)))
    return np.array(keypoints_list)




        
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/Users/samannazir/Downloads/archive/asl_alphabet_train/asl_alphabet_train')

actions = np.array([ 'Z', 'Y', 'nothing', 'W', 'V',
        'U', 'T', 'S', 'R', 'Q',
        'P', 'O', 'N', 'M','L',
        'K', 'J', 'I', 'H', 'G', 
        'F', 'E', 'D', 'C', 'B', 
        'X','space', 'A', 'del'] )

no_sequences = 30

sequence_length = 30

In [4]:
from keras.models import load_model

ASL_model = load_model('ASL_model')

In [5]:
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense



In [12]:

from keras.utils import to_categorical
from keras.models import model_from_json


# Load ASL model architecture from JSON file
with open('/Users/samannazir/model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Load the ASL model architecture
ASL_model = model_from_json(loaded_model_json)

# Load ASL model weights
ASL_model.load_weights('/Users/samannazir/model_checkpoint.h5')

# Compile the model if needed
ASL_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


colors = []
for i in range(0,20):
    colors.append((0,0,0))
print(len(colors))
def prob_viz(res, actions, input_frame, colors,threshold):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


# 1. New detection variables
sequence = []
sentence = []
accuracy=[]
predictions = []
threshold = 0.8 

cap = cv2.VideoCapture(0)
 
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        cropframe=frame[40:400,0:300]

        # print(frame.shape)
        frame=cv2.rectangle(frame,(0,40),(300,400),255,2)
        image, results = mediapipe_detection(cropframe, hands)
        # print(results)
        
        # Draw landmarks
        # draw_styled_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        #print("Key Points Shape:", keypoints.shape)
        #print("Key Points:", keypoints)

        sequence.append(keypoints)
        sequence = sequence[-30:]


        try:
            if len(sequence) == 30 and len(keypoints) > 0:

                print("Key Points new Shape:", keypoints.shape)
                res = ASL_model.predict(keypoints)[0]
                #predicted_class = np.argmax(res)    

               
                print("Predicting.model..")
                print("predicted_class:", predicted_class)
                
                #print("Extracted Key Points Shape:", keypoints.shape)
                # Make predictions using the model
                

                '''keypoints_reshaped = keypoints.reshape((100, 100, 3))
                res = ASL_model.predict(np.expand_dims(keypoints_reshaped, axis=0))[0]
                print("Prediction:", res)'''
                predicted_class = np.argmax(res)
                print(actions[predicted_class])

                print("Predicted Class:", predicted_class)
                print("Predicted Label:", actions[predicted_class])
                predictions.append(predicted_class)
              
                # Update sentence and accuracy lists
                if np.unique(predictions[-10:])[0] == predicted_class and res[predicted_class] > threshold:
                    if len(sentence) > 0:
                        if actions[predicted_class] != sentence[-1]:
                            sentence.append(actions[predicted_class])
                            accuracy.append(str(res[predicted_class] * 100))
                    else:
                        sentence.append(actions[predicted_class])
                        accuracy.append(str(res[predicted_class] * 100))

                if len(sentence) > 1:
                    sentence = sentence[-1:]
                    accuracy = accuracy[-1:]
            else:

                print("Key points array is empty.")

                # Viz probabilities
                #frame = prob_viz(res, actions, frame, colors,threshold)
        except Exception as e:
            pass
            
        cv2.rectangle(frame, (0,0), (300, 40), ((0,0,0)), -1)
        cv2.putText(frame,"Sign: "+' '.join(sentence)+' -> '.join(accuracy), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

20
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty.
Key points array is empty

In [102]:
import cv2
import numpy as np
image_path = '/Users/samannazir/Downloads/archive/asl_alphabet_train/asl_alphabet_train/space/space999.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (100, 100))  # Resize the image to match the input shape of the model
image = image.astype('float32') / 255.0  # Normalize the pixel values to be between 0 and 1

# Make predictions on the image
print()
predictions = ASL_model.predict(np.expand_dims(image, axis=0))

# Process the predictions as needed
# For example, if you are doing multi-class classification, you can get the predicted class index
predicted_class_idx = np.argmax(predictions)
# Map the class index to the corresponding class label
actions = [ 'R', 'U', 'I', 'N', 'G', 'Z', 'T', 'S', 'A', 'F', 'O', 'H', 'del', 'nothing', 'space', 'M', 'J', 'C', 'D', 'V', 'Q', 'X', 'E', 'B', 'K', 'L', 'Y', 'P', 'W']

cl = ['A', 'B', 'C', 'D', 'E',
      'F', 'G', 'H', 'I', 'J', 
      'K','L', 'M', 'N', 'O', 
      'P', 'Q', 'R', 'S', 'T', 
      'U', 'V','W', 'X', 'Y', 
      'Z', 'del', 'nothing', 'space']

cla = [ 'Z', 'Y', 'X', 'W', 'V',
        'U', 'T', 'S', 'R', 'Q',
        'P', 'O', 'N', 'M','L',
        'K', 'J', 'I', 'H', 'G', 
        'F', 'E', 'D', 'C', 'B', 
        'A','space', 'nothing', 'del']          
predicted_class_label = actions[predicted_class_idx]
predicted_cl = cl[predicted_class_idx]
predicted_cla = cla[predicted_class_idx]

print("Predicted Label:", predicted_class_label)
print("Predicted Label cl :", predicted_cl)
print("Predicted Label cla :", predicted_cla)

(1, 100, 100, 3)
1/1 [==============================] - 0s 146ms/step
Predicted Label: P
Predicted Label cl : nothing
Predicted Label cla : nothing


In [42]:
cap.release()
cv2.destroyAllWindows()